In [23]:
# sliced 3D real image
import warnings
warnings.filterwarnings("ignore")


import numpy as np
from concurrent.futures import ProcessPoolExecutor
import time
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import tensorly as tl

#Debugging import
import importlib
var = 'TensorDecisionTreeRegressorP' #the published version of code
package = importlib.import_module(var)
for name, value in package.__dict__.items():
    if not name.startswith("__"):
        globals()[name] = value

from TensorDecisionTreeRegressorP import *

import os
import nibabel as nib
import numpy as np
import matplotlib as plt
import pandas as pd
from sklearn.model_selection import train_test_split

# File path to the CSV file
csv_file = '/Users/zhihaochen/Documents/CommenDesktop/RICE/MyProject/Bayes_Tensor_Tree/3D-images/ADNIData.csv'
#csv_file = '/home/zc56/Bayes_Tensor_Tree/3D-images/ADNIData.csv'
# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file)

# Extract the 'ADAS11' column as the y variable
y_variable = df['RAVLT_forgetting'].values  # Convert to NumPy array if needed

# Print the shape and first few values of the y variable
print(f"y variable shape: {y_variable.shape}")
print(f"First 5 values of y variable: {y_variable[:5]}")



directory = '/Users/zhihaochen/Documents/CommenDesktop/RICE/MyProject/Bayes_Tensor_Tree/3D-images/3D-Images/bl'
#directory = '/home/zc56/Bayes_Tensor_Tree/3D-images/3D-Images/bl'
# Initialize a list to hold 3D images
# Initialize a list to hold 3D images
image_list = []

# Loop over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.nii.gz'):
        file_path = os.path.join(directory, filename)
        
        # Load the NIfTI file
        img = nib.load(file_path)
        
        # Get the data as a NumPy array
        data = img.get_fdata()
        
        # Append the 3D image data to the list
        image_list.append(data)
        
        # Print the filename and data shape
        #print(f"Loaded {filename}, shape: {data.shape}")

# Stack the 3D images along a new axis to create a 4D tensor
if image_list:
    image_tensor = np.stack(image_list, axis=0)
    #print(f"Stacked 4D tensor shape: {image_tensor.shape}")
else:
    print("No images loaded.")


X_train, X_test, y_train, y_test = train_test_split(image_tensor, y_variable, test_size=0.3, random_state=42)
y_train = np.nan_to_num(y_train, nan=0.0)
# Print the shapes of the training and testing data
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

# Remove rows from X_train and y_train where y_train has NaN values
non_nan_train_indices = ~np.isnan(y_train)
X_train = X_train[non_nan_train_indices]
y_train = y_train[non_nan_train_indices]

# Remove rows from X_test and y_test where y_test has NaN values
non_nan_test_indices = ~np.isnan(y_test)
X_test = X_test[non_nan_test_indices]
y_test = y_test[non_nan_test_indices]

# Print the new shapes after removing NaN values
print(f"X_train_cleaned shape: {X_train.shape}")
print(f"y_train_cleaned shape: {y_train.shape}")
print(f"X_test_cleaned shape: {X_test.shape}")
print(f"y_test_cleaned shape: {y_test.shape}")


y variable shape: (818,)
First 5 values of y variable: [3. 5. 1. 4. 3.]
X_train shape: (572, 48, 48, 48)
y_train shape: (572,)
X_test shape: (246, 48, 48, 48)
y_test shape: (246,)
X_train_cleaned shape: (572, 48, 48, 48)
y_train_cleaned shape: (572,)
X_test_cleaned shape: (245, 48, 48, 48)
y_test_cleaned shape: (245,)


In [24]:
print(y_variable)

[ 3.  5.  1.  4.  3.  9. 10.  5.  5.  4.  3.  5.  1.  6.  4.  8.  5.  4.
  3.  5.  1.  6.  8.  5.  1.  2.  4.  6.  6. 10.  6.  7.  5.  7.  5.  6.
  3.  3.  6.  4.  4.  1.  3.  3.  5.  5.  0.  2.  7.  4.  4.  2.  5.  9.
  9.  5.  9.  6.  7.  4.  4.  2.  5.  3.  5.  3.  5.  2.  4.  3.  6.  8.
  2.  3.  4.  5. 13.  5.  2.  7.  4.  6.  4.  4.  3.  5.  6.  1.  2.  7.
  4.  2.  5.  9.  4.  7.  6.  0.  3.  2.  6.  4.  3.  4.  5.  3.  4.  5.
  2.  4.  0.  5.  6.  6.  3.  4.  8.  6.  5.  8.  5.  0.  6.  5.  2.  8.
  3.  7.  7.  1.  4.  5.  4.  3.  4.  8.  1.  1.  8.  2.  1.  8.  4.  6.
  4.  6.  0.  7.  2.  6.  1.  5.  3.  5.  4.  5.  6.  2.  1.  8.  3.  6.
  4.  3.  4.  5.  5.  2.  6.  3.  6.  3.  3. -1.  4.  5.  3.  0.  4.  0.
  6.  5.  8.  0.  3.  6.  5.  4.  7.  6.  3.  2.  2.  8.  4.  3.  4.  7.
  3.  1.  5.  6.  4.  5.  2. 10.  6.  1.  6.  2.  4.  8. 10.  2.  8.  1.
  2.  4.  2.  4.  6.  4.  7.  5.  4.  7.  3.  5.  5.  5.  5.  4.  3.  6.
 12.  5.  4.  6.  8.  5.  6.  0.  6.  2.  6.  4.  5

In [20]:
import os
import pandas as pd
import numpy as np
import nibabel as nib

# Load the CSV file containing PTID and target variable
csv_file = '/Users/zhihaochen/Documents/CommenDesktop/RICE/MyProject/Bayes_Tensor_Tree/3D-images/ADNIData.csv'
df = pd.read_csv(csv_file)

# Extract PTID and target values from the DataFrame
ptid_csv = df['PTID'].values
y_variable = df['RAVLT_forgetting'].values

# Directory containing the image files
directory = '/Users/zhihaochen/Documents/CommenDesktop/RICE/MyProject/Bayes_Tensor_Tree/3D-images/3D-Images/bl'

# Initialize lists for storing matched images and corresponding y values
matched_images = []
matched_y_values = []

# Loop through the image files and match by PTID
for filename in os.listdir(directory):
    if filename.endswith('.nii.gz'):
        # Extract PTID from the filename (e.g., '002_S_0295.nii.gz' -> '002_S_0295')
        ptid_image = filename.split('.')[0]  # Keep the '002_S_0295' format with underscores
        
        # Print the extracted PTID from filename (for debugging)
        # print(f"Extracted PTID from filename '{filename}': {ptid_image}")
        
        # Find the index of this PTID in the CSV data
        if ptid_image in ptid_csv:
            index = np.where(ptid_csv == ptid_image)[0][0]  # Get the index of the matching PTID
            y_value = y_variable[index]  # Get the corresponding target value

            # Load the image
            file_path = os.path.join(directory, filename)
            img = nib.load(file_path)
            data = img.get_fdata()

            # Append the image data and the corresponding y_value to the lists
            matched_images.append(data)
            matched_y_values.append(y_value)
        else:
            print(f"PTID '{ptid_image}' not found in the CSV file.")

# Convert the matched images and y values into arrays
if matched_images:
    image_tensor = np.stack(matched_images, axis=0)
    y_variable = np.array(matched_y_values)
    print(f"Aligned image tensor shape: {image_tensor.shape}")
    print(f"Aligned y variable shape: {y_variable.shape}")
else:
    print("No images matched.")


Aligned image tensor shape: (818, 48, 48, 48)
Aligned y variable shape: (818,)


In [25]:
import os
import pandas as pd
import numpy as np
import nibabel as nib

# Load the CSV file containing PTID and target variable
csv_file = '/Users/zhihaochen/Documents/CommenDesktop/RICE/MyProject/Bayes_Tensor_Tree/3D-images/ADNIData.csv'
df = pd.read_csv(csv_file)

# Extract PTID and target values from the DataFrame
ptid_csv = df['PTID'].values
y_variable = df['RAVLT_forgetting'].values

# Directory containing the image files
directory = '/Users/zhihaochen/Documents/CommenDesktop/RICE/MyProject/Bayes_Tensor_Tree/3D-images/3D-Images/bl'

# Initialize lists for storing matched images and corresponding y values
matched_images = []
matched_y_values = []

# Loop through the image files and match by PTID
for filename in os.listdir(directory):
    if filename.endswith('.nii.gz'):
        # Extract PTID from the filename (e.g., '002_S_0295.nii.gz' -> '002_S_0295')
        ptid_image = filename.split('.')[0]  # Keep the '002_S_0295' format with underscores
        
        # Find the index of this PTID in the CSV data
        if ptid_image in ptid_csv:
            index = np.where(ptid_csv == ptid_image)[0][0]  # Get the index of the matching PTID
            y_value = y_variable[index]  # Get the corresponding target value

            # Skip if the corresponding y_value is NaN
            if np.isnan(y_value):
                print(f"Skipping PTID '{ptid_image}' because y_value is NaN.")
                continue

            # Load the image
            file_path = os.path.join(directory, filename)
            img = nib.load(file_path)
            data = img.get_fdata()

            # Append the image data and the corresponding y_value to the lists
            matched_images.append(data)
            matched_y_values.append(y_value)
        else:
            print(f"PTID '{ptid_image}' not found in the CSV file.")

# Convert the matched images and y values into arrays
if matched_images:
    image_tensor = np.stack(matched_images, axis=0)
    y_variable = np.array(matched_y_values)
    print(f"Aligned image tensor shape: {image_tensor.shape}")
    print(f"Aligned y variable shape: {y_variable.shape}")
else:
    print("No images matched.")


Skipping PTID '099_S_0372' because y_value is NaN.
Skipping PTID '073_S_0311' because y_value is NaN.
Skipping PTID '141_S_0853' because y_value is NaN.
Skipping PTID '141_S_1094' because y_value is NaN.
Aligned image tensor shape: (814, 48, 48, 48)
Aligned y variable shape: (814,)


In [27]:
os.listdir(directory)

['098_S_0269.nii.gz',
 '002_S_0619.nii.gz',
 '029_S_0999.nii.gz',
 '052_S_0951.nii.gz',
 '068_S_0109.nii.gz',
 '133_S_0771.nii.gz',
 '127_S_1419.nii.gz',
 '057_S_0474.nii.gz',
 '003_S_1059.nii.gz',
 '128_S_0611.nii.gz',
 '137_S_0481.nii.gz',
 '136_S_0695.nii.gz',
 '032_S_0214.nii.gz',
 '041_S_1418.nii.gz',
 '094_S_0692.nii.gz',
 '098_S_0884.nii.gz',
 '036_S_1135.nii.gz',
 '011_S_0003.nii.gz',
 '006_S_0731.nii.gz',
 '128_S_1406.nii.gz',
 '013_S_1161.nii.gz',
 '062_S_1294.nii.gz',
 '033_S_1309.nii.gz',
 '006_S_0498.nii.gz',
 '023_S_0126.nii.gz',
 '082_S_0928.nii.gz',
 '098_S_0896.nii.gz',
 '082_S_0640.nii.gz',
 '082_S_1377.nii.gz',
 '018_S_0087.nii.gz',
 '116_S_1271.nii.gz',
 '123_S_0106.nii.gz',
 '021_S_0984.nii.gz',
 '002_S_0955.nii.gz',
 '035_S_0204.nii.gz',
 '035_S_0555.nii.gz',
 '128_S_1430.nii.gz',
 '073_S_0909.nii.gz',
 '009_S_0862.nii.gz',
 '032_S_0187.nii.gz',
 '100_S_0190.nii.gz',
 '031_S_0351.nii.gz',
 '133_S_0493.nii.gz',
 '023_S_0061.nii.gz',
 '014_S_0558.nii.gz',
 '013_S_02

In [22]:
print(y_variable)

[ 7.  4.  4.  3.  2.  3.  4.  9.  5.  6.  5.  6.  6.  7.  6. 10.  5.  4.
  2.  0.  3.  4.  5.  3.  7.  3.  1.  2.  8.  6.  3.  4.  2.  3.  6.  7.
  4.  7.  2.  3.  3.  8.  3.  2.  8.  8.  2. -1.  5.  6.  5.  5.  3.  7.
  6.  5.  1.  6.  6.  7.  5.  5.  5.  1.  3.  9.  5.  2.  1.  5.  4.  8.
  4.  2.  5.  7.  3.  9.  2.  7.  5.  4.  4.  6.  8.  3.  6.  3.  2.  5.
  7.  6.  2.  0.  5.  6.  6.  2.  6.  7.  5.  6.  7.  3.  6.  4.  0.  2.
  4.  7.  5.  6.  4.  5.  4.  7.  5.  2.  5.  7.  5.  4.  5.  6.  5.  1.
  4.  5.  2.  6.  3.  6.  0.  3.  1.  2.  5.  5.  5.  5.  5.  3.  3.  2.
  7.  3.  4.  2.  1.  6.  4.  6.  5.  1.  4.  5.  4.  1.  4.  5.  8.  1.
  5.  8.  2.  6.  3.  6.  3.  2.  3.  0.  4.  7.  3.  5.  6.  8.  5.  4.
  8. -2.  5.  5.  4.  5.  5. -2.  4.  4.  0.  9.  5.  2.  5.  8.  5.  4.
  4.  3.  3.  4.  1.  5.  5.  6.  1.  9.  6.  7.  7.  2.  6.  3.  8.  6.
  5.  5.  5.  5.  3. 10.  1.  7.  5.  4.  5.  6.  2.  4.  1.  4.  4. 11.
  0.  9.  3. -1.  3.  9.  6.  1.  6.  4.  4.  2.  0

In [8]:
const_array = np.load('const_list.npy', allow_pickle=True)
const_list = list(map(tuple, const_array))

print(X_train.shape,y_train.shape)
model  =  TensorDecisionTreeRegressor(max_depth=10, min_samples_split=12,split_method='variance_LS', split_rank=4, CP_reg_rank=3, Tucker_reg_rank=3, n_mode=4, const_array=const_list)
model.sample_rate  =  .1
model.fit(X_train,y_train)

predictions = model.predict(X_train,regression_method='mean')
print(f"mean train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train,regression_method='cp')
print(f"CP train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train,regression_method='tucker')
print(f"Tucker train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train)) 

predictions = model.predict(X_test,regression_method='mean')
print(f"mean test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test,regression_method='cp')
print(f"CP test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test,regression_method='tucker')
print(f"Tucker test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))

(572, 48, 48, 48) (572,)
mean train RSE:  0.1620723842100913
CP train RSE:  0.00020723882451839228
Tucker train RSE:  0.0006394023838957846
mean test RSE:  1.6886257274045864
CP test RSE:  1.890609550149363
Tucker test RSE:  1.7383403989394204


In [9]:
model.print_tree()

  if X[:, 25 , 10 , 24 ] <=  2.4282201724452572
   if X[:, 35 , 18 , 7 ] <=  0.7400850550202299
     if X[:, 25 , 33 , 5 ] <=  2.314671340114192
         if X[:, 26 , 27 , 18 ] <=  0.5160187291793334
                 if X[:, 30 , 30 , 20 ] <=  2.4199839247597588
                                 if X[:, 30 , 24 , 43 ] <=  1.7220184477892788
                                                                 if X[:, 33 , 26 , 41 ] <=  1.2471299171447754
                                                                                                                                 has  0  child nodes, and  9  samples.
                                                                 else: # if X[:, 33 , 26 , 41 ] >  1.2471299171447754
                                                                                                                                 has  0  child nodes, and  4  samples.
                                 else: # if X[:, 30 , 24 , 43 ] >  1.7220184477892788
         

In [26]:
X_train, X_test, y_train, y_test = train_test_split(image_tensor, y_variable, test_size=0.3, random_state=42)
const_array = np.load('const_list.npy', allow_pickle=True)
const_list = list(map(tuple, const_array))

print(X_train.shape,y_train.shape)
model  =  TensorDecisionTreeRegressor(max_depth=10, min_samples_split=12,split_method='variance_LS', split_rank=4, CP_reg_rank=3, Tucker_reg_rank=3, n_mode=4, const_array=const_list)
model.sample_rate  =  .1
model.fit(X_train,y_train)

predictions = model.predict(X_train,regression_method='mean')
print(f"mean train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train,regression_method='cp')
print(f"CP train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train,regression_method='tucker')
print(f"Tucker train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train)) 

predictions = model.predict(X_test,regression_method='mean')
print(f"mean test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test,regression_method='cp')
print(f"CP test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test,regression_method='tucker')
print(f"Tucker test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))

(569, 48, 48, 48) (569,)
mean train RSE:  0.15375951611876612
CP train RSE:  0.00024689560310563294
Tucker train RSE:  0.000332815768678579
mean test RSE:  1.7515030889986578
CP test RSE:  2.125251701026327
Tucker test RSE:  2.1713708708244437


In [28]:
from sklearn.model_selection import KFold
import numpy as np

# Initialize variables to store results
train_rse_mean, train_rse_cp, train_rse_tucker = [], [], []
test_rse_mean, test_rse_cp, test_rse_tucker = [], [], []

# Initialize 10-fold cross-validation
kf = KFold(n_splits=10)

# Loop over the K folds, performing cross-validation on both image_tensor and y_variable
for train_index, val_index in kf.split(image_tensor):
    # Split image_tensor and y_variable into training and validation sets
    X_train_fold, X_val_fold = image_tensor[train_index], image_tensor[val_index]
    y_train_fold, y_val_fold = y_variable[train_index], y_variable[val_index]

    # Initialize and fit the model using the training fold
    model = TensorDecisionTreeRegressor(max_depth=10, min_samples_split=12, split_method='variance_LS', split_rank=4,
                                        CP_reg_rank=3, Tucker_reg_rank=3, n_mode=4, const_array=const_list)
    model.sample_rate = 0.1
    model.fit(X_train_fold, y_train_fold)

    # Predictions for training set
    pred_train_mean = model.predict(X_train_fold, regression_method='mean')
    pred_train_cp = model.predict(X_train_fold, regression_method='cp')
    pred_train_tucker = model.predict(X_train_fold, regression_method='tucker')

    # Calculate training RSEs using the training fold
    train_rse_mean.append(np.mean((pred_train_mean - y_train_fold) ** 2) / np.var(y_train_fold))
    train_rse_cp.append(np.mean((pred_train_cp - y_train_fold) ** 2) / np.var(y_train_fold))
    train_rse_tucker.append(np.mean((pred_train_tucker - y_train_fold) ** 2) / np.var(y_train_fold))

    # Predictions for validation set
    pred_val_mean = model.predict(X_val_fold, regression_method='mean')
    pred_val_cp = model.predict(X_val_fold, regression_method='cp')
    pred_val_tucker = model.predict(X_val_fold, regression_method='tucker')

    # Calculate validation RSEs using the validation fold
    test_rse_mean.append(np.mean((pred_val_mean - y_val_fold) ** 2) / np.var(y_val_fold))
    test_rse_cp.append(np.mean((pred_val_cp - y_val_fold) ** 2) / np.var(y_val_fold))
    test_rse_tucker.append(np.mean((pred_val_tucker - y_val_fold) ** 2) / np.var(y_val_fold))

# Function to compute stats (max, min, mean, sd)
def compute_stats(rse_list):
    return {
        'max': np.max(rse_list),
        'min': np.min(rse_list),
        'mean': np.mean(rse_list),
        'std': np.std(rse_list)
    }

# Compute stats for training RSEs
train_stats_mean = compute_stats(train_rse_mean)
train_stats_cp = compute_stats(train_rse_cp)
train_stats_tucker = compute_stats(train_rse_tucker)

# Compute stats for validation RSEs
val_stats_mean = compute_stats(test_rse_mean)
val_stats_cp = compute_stats(test_rse_cp)
val_stats_tucker = compute_stats(test_rse_tucker)

# Print the stats
print("Training RSE Stats (Mean Regression):", train_stats_mean)
print("Training RSE Stats (CP Regression):", train_stats_cp)
print("Training RSE Stats (Tucker Regression):", train_stats_tucker)

print("Validation RSE Stats (Mean Regression):", val_stats_mean)
print("Validation RSE Stats (CP Regression):", val_stats_cp)
print("Validation RSE Stats (Tucker Regression):", val_stats_tucker)



Training RSE Stats (Mean Regression): {'max': np.float64(0.1832054688142523), 'min': np.float64(0.13596358322423754), 'mean': np.float64(0.15723241801028448), 'std': np.float64(0.016470385498109395)}
Training RSE Stats (CP Regression): {'max': np.float64(0.00023181665416001165), 'min': np.float64(0.00019490104368707133), 'mean': np.float64(0.00021171132155271274), 'std': np.float64(1.1646933159591953e-05)}
Training RSE Stats (Tucker Regression): {'max': np.float64(0.001761570150601049), 'min': np.float64(0.00027496251891602665), 'mean': np.float64(0.0010241151401822772), 'std': np.float64(0.0006016953364013685)}
Validation RSE Stats (Mean Regression): {'max': np.float64(2.602066219371753), 'min': np.float64(1.5214331836934214), 'mean': np.float64(2.0639806914863703), 'std': np.float64(0.3141827191281695)}
Validation RSE Stats (CP Regression): {'max': np.float64(3.3915191216777023), 'min': np.float64(1.6967539983105975), 'mean': np.float64(2.5402227620698046), 'std': np.float64(0.460519